# OpenSearch Notebook

Notebook with the propuse to test and improve the openSearch queries to index and read data.

Done by Víctor Gutiérrez Tovar

# Setup

In [5]:
import sys
import json
sys.path.append("..")
import os
os.chdir("./..")
from utils import get_opensearch_client
os.chdir("./lab")

os_client = get_opensearch_client()

# Read or view Indexes

In [3]:
response = os_client.cat.indices(format="json")

for index in response:
    print(index["index"])


test-index
.opensearch-observability
.plugins-ml-config
data
security-auditlog-2024.03.26
security-auditlog-2024.03.17
security-auditlog-2024.03.25
.opendistro_security


# Read docs from a index

## Test index

In [11]:
def get_all_docs(index_name):
    # Execute the search query to get the first 1000 documents
    response = os_client.search(
        index=index_name,
        body={"query": {"match_all": {}}},
        size=1000
    )

    # Iterate through the hits and print document details
    for doc in response["hits"]["hits"]:
        print(doc["_id"], doc["_source"])

# Example usage
index_name = "test-index"
get_all_docs(index_name)

1 {'action': 'index', 'title': 'El barco velero', 'content': 'Content of document 1'}
2 {'action': 'index', 'title': 'Hawai', 'content': 'Content of document 2'}
3 {'action': 'index', 'title': 'Document 3 Updated', 'content': 'Updated content of document 3'}
4 {'action': 'index', 'title': 'Document 4 Updated', 'content': 'Updated content of document 3'}


## Data index (the default index)

In [23]:
def get_all_docs(index_name):
    # Execute the search query to get the first 1000 documents
    response = os_client.search(
        index=index_name,
        body={"query": {"match_all": {}}},
        size=1000,
        ignore=[400, 404]
    )
    # print(response)
    if(not "error" in response):
        # Iterate through the hits and print document details
        for doc in response["hits"]["hits"]:
            print(doc["_id"], doc["_source"])

# Example usage
index_name = "data"
get_all_docs(index_name)

../data/test.csv-2 {'id': '2', 'name': 'Ibuprofen', 'description': 'Used to relieve pain from various conditions such as headache, dental pain, menstrual cramps, muscle aches, or arthritis.', 'use_cases': 'Pain, inflammation, fever.', 'file_name': 'test'}
../data/test.csv-3 {'id': '3', 'name': 'Amoxicillin', 'description': 'An antibiotic used to treat a number of bacterial infections.', 'use_cases': 'Ear infections, strep throat, pneumonia, urinary tract infections.', 'file_name': 'test'}
../data/test.csv-4 {'id': '4', 'name': 'Cetirizine', 'description': 'An antihistamine used to relieve allergy symptoms such as watery eyes, runny nose, itching eyes/nose, sneezing, hives, and itching.', 'use_cases': 'Hay fever, allergies.', 'file_name': 'test'}
../data/test.csv-5 {'id': '5', 'name': 'Metformin', 'description': 'A medication used to treat type 2 diabetes.', 'use_cases': 'Type 2 diabetes management.', 'file_name': 'test'}
../data/test.csv-1 {'id': '1', 'name': 'Paracetamol', 'descriptio

# Delete index

In [6]:
index_name = "data"

response = os_client.indices.delete(index=index_name, ignore=[400, 404])

# Search engine

In [7]:
index_name = "data"

body = {
  "query": {
    "query_string": {
      "query": "pain",
      "default_field": "*"
    }
  }
}

response = os_client.search(body, index=index_name)

# Iterate through the hits and print document details
for doc in response["hits"]["hits"]:
    print(f"id: {doc["_id"]}, score: {doc["_score"]}")
    print(json.dumps(doc["_source"], indent=2))


id: ../data/test.csv-6, score: 1.9966218
{
  "id": "6",
  "name": "Acetaminophen",
  "description": "Pain reliever and fever reducer.",
  "use_cases": "Pain, fever.",
  "file_name": "test"
}
id: ../data/test.csv-1, score: 1.8773284
{
  "id": "1",
  "name": "Paracetamol",
  "description": "Pain reliever and a fever reducer.",
  "use_cases": "Mild to moderate pain, fever.",
  "file_name": "test"
}
id: ../data/test.csv-19, score: 1.8773284
{
  "id": "19",
  "name": "Gabapentin",
  "description": "Used for nerve pain and seizures.",
  "use_cases": "Nerve pain, seizures.",
  "file_name": "test"
}
id: ../data/test.csv-2, score: 1.6985979
{
  "id": "2",
  "name": "Ibuprofen",
  "description": "Used to relieve pain from various conditions such as headache, dental pain, menstrual cramps, muscle aches, or arthritis.",
  "use_cases": "Pain, inflammation, fever.",
  "file_name": "test"
}
id: ../data/test.csv-8, score: 1.6350331
{
  "id": "8",
  "name": "Naproxen",
  "description": "An anti-inflamm